# RAG com LlamaIndex, OpenAI e banco de dados vetorial MongoDB

Este notebook implementa um sistema RAG completo utilizando o stack de IA `POLM` (Python, OpenAI, LlamaIndex, MongoDB). O stack de IA, também chamado de GenAI stack, é composto por modelos, bancos de dados, bibliotecas e frameworks que permitem a construção de aplicações modernas com capacidades de IA generativa.

Neste projeto, utilizamos o modelo `text-embedding-3-small` da `OpenAI` para gerar embeddings, o `LlamaIndex` como estrutura de orquestração, e o `MongoDB`, que atua tanto como banco de dados operacional quanto como armazenamento vetorial.


### Estrutura do Projeto:
1. Carregamento do Dataset: O conjunto de dados é carregado a partir do `Hugging Face`.
2. Geração de Embeddings: Embeddings são gerados utilizando o modelo de embeddings da `OpenAI`.
3. Configuração do Banco de Dados Vetorial: Um banco de dados vetorial é configurado no `MongoDB` para armazenar os embeddings.
4. Estabelecimento de Conexão: Uma conexão segura é estabelecida com o banco de dados `MongoDB`.
5. Criação de Índice de Busca Vetorial: Um índice de busca vetorial é criado para permitir consultas eficientes.

### Bibliotecas Necessárias:

`LlamaIndex`: Framework de dados que facilita a integração de fontes de dados (arquivos, PDFs, sites) com LLMs como OpenAI e Cohere e LLMs de código aberto (como Llama).

`LlamaIndex` para `MongoDB`: Extensão do LlamaIndex que fornece métodos para conectar e interagir com o MongoDB Atlas.

`LlamaIndex` para `OpenAI`: Extensão do LlamaIndex que inclui os métodos necessários para acessar os modelos de embeddings da OpenAI.

`PyMongo`: Biblioteca Python utilizada para conectar-se ao MongoDB, realizar consultas em coleções e manipular documentos.

`Hugging Face datasets`: biblioteca Hugging Face que contém varios datasets prontos para uso.

`Pandas`:  para processamento e análise eficientes de dados tabulares usando Python.

In [ ]:
%pip install llama-index
%pip install llama-index-vector-stores-mongodb
%pip install llama-index-embeddings-openai
%pip install pymongo
%pip install datasets
%pip install pandas


Utilizamos as bibliotecas `getpass` e `os` para armazenar a chave da OpenAI de forma segura, sem que ela seja exibida na tela.

In [1]:
import getpass
import os

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPEN API KEY:")

Baixamos os dados de filmes através de `Hugging face`:

In [3]:
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies

ds = load_dataset("MongoDB/embedded_movies")


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Convertemos para dataframe

dataset_df=pd.DataFrame(ds['train'])

dataset_df.head()

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic,plot_embedding
0,Young Pauline is left a lot of money when her ...,199.0,[Action],Young Pauline is left a lot of money when her ...,"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...",[USA],https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,0,None,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.0007293965299999999, -0.026834568000000003,..."
1,A penniless young man tries to save an heiress...,22.0,"[Comedy, Short, Action]",As a penniless man worries about how he will m...,"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],[USA],https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.022837115, -0.022941574000000003, 0.014937..."
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"[Action, Adventure, Drama]","Michael ""Beau"" Geste leaves England in disgrac...",[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...",[USA],None,[English],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,0,None,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.00023330492999999998, -0.028511643000000003..."
3,"Seeking revenge, an athletic young man joins t...",88.0,"[Adventure, Action]",A nobleman vows to avenge the death of his fat...,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...",[USA],https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,1,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[-0.005927917, -0.033394486, 0.0015323418, -0...."
4,An irresponsible young millionaire changes his...,58.0,"[Action, Comedy, Romance]","The Uptown Boy, J. Harold Manners (Lloyd) is a...",[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...",[USA],https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0059373598, -0.026604708, -0.0070914757000..."


In [5]:
dataset_df.shape

(1500, 18)

In [6]:
dataset_df.columns

Index(['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers',
       'countries', 'poster', 'languages', 'cast', 'title',
       'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic',
       'plot_embedding'],
      dtype='object')

O dataset contém informações de 1500 filmes, incluindo enredo (`plot`) e enredo completo (`fullplot`), são features que serão utilizados no processo de embeddings. Precisamos garantir que não contenham valores nulos. Além disso, vamos apagar `plot_embedding`, já que criaremos nossos próprios embeddings posteriormente.

In [7]:
# Removemos valores ausentes nas colunas plot e fullplot 

dataset_df=dataset_df.dropna(subset=['plot','fullplot'])

print("\nNumber of missing values in each column after removal:")

print(dataset_df.isnull().sum())

# Removemos a coluna plot_embedding 

dataset_df=dataset_df.drop(columns=['plot_embedding'])

dataset_df.head()


Number of missing values in each column after removal:
plot                    0
runtime                14
genres                  0
fullplot                0
directors              12
writers                13
countries               0
poster                 78
languages               1
cast                    1
title                   0
num_mflix_comments      0
rated                 279
imdb                    0
awards                  0
type                    0
metacritic            893
plot_embedding          1
dtype: int64


,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic
0,Young Pauline is left a lot of money when her ...,199.0,[Action],Young Pauline is left a lot of money when her ...,"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...",[USA],https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,0,None,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN
1,A penniless young man tries to save an heiress...,22.0,"[Comedy, Short, Action]",As a penniless man worries about how he will m...,"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],[USA],https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"[Action, Adventure, Drama]","Michael ""Beau"" Geste leaves England in disgrac...",[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...",[USA],None,[English],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,0,None,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN
3,"Seeking revenge, an athletic young man joins t...",88.0,"[Adventure, Action]",A nobleman vows to avenge the death of his fat...,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...",[USA],https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,1,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN
4,An irresponsible young millionaire changes his...,58.0,"[Action, Comedy, Romance]","The Uptown Boy, J. Harold Manners (Lloyd) is a...",[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...",[USA],https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN


In [8]:
dataset_df.shape

(1452, 17)

Após a limpeza, o dataset contém 17 features e 1452 entradas. Para executar os próximos passos, é necessário ter acesso à API paga da OpenAI (no mínimo o tier 1). No nível gratuito da API, o modelo `text-embedding-3-small` permite 3000 tokens por minuto e um máximo de 200 solicitações por dia. No tier 1, não há limite diário de acessos. Como estou utilizando o acesso gratuito, vou reduzir o dataset para 100 entradas, a fim de não exceder o limite de 200 solicitações por dia.

In [9]:
df = dataset_df.head(100)

Vamos configurar o ambiente para utilizar o LlamaIndex com o modelo de linguagem e embeddings da OpenAI. Usaremos a classe `OpenAIEmbedding`, que faz parte do módulo `llama_index.embeddings`, para gerar embeddings a partir de textos. A classe `OpenAIEmbedding` recebe dois parâmetros: o nome do modelo de embeddings (`text-embedding-3-small` no nosso caso) e a dimensão dos embeddings (256).

Em seguida, converteremos nosso dataframe do Pandas para uma lista de dicionários no formato `JSON`, utilizando a biblioteca `json`. O argumento `orient='records'` indica que cada linha do dataframe será convertida em um dicionário, resultando em uma lista de dicionários.

In [10]:
from llama_index.core.settings import Settings # Armazena as configurações globais para a execução dos modelos
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model=OpenAIEmbedding(model="text-embedding-3-small",dimensions=256)

llm=OpenAI()

Settings.llm=llm

Settings.embed_model=embed_model

import json
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

# Convert the DataFrame to a JSON string representation
documents_json = df.to_json(orient='records')
# Load the JSON string into a Python list of dictionaries
documents_list = json.loads(documents_json)

O próximo passo é converter cada dicionário em documentos manualmente. Os documentos no LlamaIndex contêm informações adicionais, como metadados, que são utilizadas no processamento e na ingestão em um pipeline de RAG.

In [11]:
llama_documents = []

for document in documents_list:

  # Value for metadata must be one of (str, int, float, None)
  document["writers"] = json.dumps(document["writers"])
  document["languages"] = json.dumps(document["languages"])
  document["genres"] = json.dumps(document["genres"])
  document["cast"] = json.dumps(document["cast"])
  document["directors"] = json.dumps(document["directors"])
  document["countries"] = json.dumps(document["countries"])
  document["imdb"] = json.dumps(document["imdb"])
  document["awards"] = json.dumps(document["awards"])


  # Create a Document object with the text and excluded metadata for llm and embedding models
  llama_document = Document(
      text=document["fullplot"],
      metadata=document,
      excluded_llm_metadata_keys=["fullplot", "metacritic"],
      excluded_embed_metadata_keys=["fullplot", "metacritic", "poster", "num_mflix_comments", "runtime", "rated"],
      metadata_template="{key}=>{value}",
      text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
      )

  llama_documents.append(llama_document)

# Observing an example of what the LLM and Embedding model receive as input
print(
    "\nThe LLM sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.LLM),
)
print(
    "\nThe Embedding model sees this: \n",
    llama_documents[0].get_content(metadata_mode=MetadataMode.EMBED),
)


The LLM sees this: 
 Metadata: plot=>Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ...
runtime=>199.0
genres=>["Action"]
directors=>["Louis J. Gasnier", "Donald MacKenzie"]
writers=>["Charles W. Goddard (screenplay)", "Basil Dickey (screenplay)", "Charles W. Goddard (novel)", "George B. Seitz", "Bertram Millhauser"]
countries=>["USA"]
poster=>https://m.media-amazon.com/images/M/MV5BMzgxODk1Mzk2Ml5BMl5BanBnXkFtZTgwMDg0NzkwMjE@._V1_SY1000_SX677_AL_.jpg
languages=>["English"]
cast=>["Pearl White", "Crane Wilbur", "Paul Panzer", "Edward Jos\u00e8"]
title=>The Perils of Pauline
num_mflix_comments=>0
rated=>None
imdb=>{"id": 4465, "rating": 7.6, "votes": 744}
awards=>{"nominations": 0, "text": "1 win.", "wins": 1}
type=>movie
-----
Content: Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as

Verificamos o conteudo de um documento:

In [12]:
llama_documents[25]

Document(id_='f47b5c27-92d8-43bf-81f3-c59f4bcaf96a', embedding=None, metadata={'plot': 'A night club owner becomes infatuated with a torch singer and frames his best friend/manager for embezzlement when the chanteuse falls in love with him.', 'runtime': 95.0, 'genres': '["Action", "Drama", "Film-Noir"]', 'fullplot': 'Jefty, owner of a roadhouse in a backwoods town, hires sultry, tough-talking torch singer Lily Stevens against the advice of his manager Pete Morgan. Jefty is smitten with Lily, who in turn exerts her charms on the more resistant Pete. When Pete finally falls for her and she turns down Jefty\'s marriage proposal, they must face Jefty\'s murderous jealousy and his twisted plots to "punish" the two.', 'directors': '["Jean Negulesco"]', 'writers': '["Edward Chodorov (screen play)", "Margaret Gruen (story)", "Oscar Saul (story)"]', 'countries': '["USA"]', 'poster': 'https://m.media-amazon.com/images/M/MV5BMjc1ZTNkM2UtYzY3Yi00ZWZmLTljYmEtNjYxZDNmYzk2ZjkzXkEyXkFqcGdeQXVyMjUxODE0

A etapa final do processamento, antes de armazenar os dados no banco de dados vetorial do MongoDB, é converter a lista de documentos do LlamaIndex em sentenças menores, ou `nodes`. Os `nodes` representam unidades em estruturas complexas, como árvores ou grafos, enquanto tokens são as menores unidades de texto manipuladas.

Utilizamos o módulo `SentenceSplitter` do LlamaIndex para dividir os documentos em nodes e, em seguida, geramos embeddings para cada um desses nodes. 



In [13]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(llama_documents)

for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding
     

Antes de prosseguir, precisaremos criar o cluster de banco de dados no MongoDB Atlas, seguindo o manual disponível em https://www.mongodb.com/docs/guides/atlas/cluster/.

Após criar o cluster, crie o banco de dados e a coleção dentro do cluster clicando em `Create Database`. O banco de dados será chamado `movies`, e a coleção será chamada `movies_records`.


Para conectar ao cluster, forneça a `URI` (connection string). É importante que seu endereço IP seja adicionado à lista de permissões para autorizar o acesso ao seu cluster Atlas.

In [14]:
os.environ["URI"] = getpass.getpass("URI:")

In [15]:
from pymongo.mongo_client import MongoClient
from pymongo.errors import ConnectionFailure

# Establishing connection
try:
    uri = os.environ["URI"]
    connect = MongoClient(uri)
    print("MongoDB cluster is reachable")
    print(connect)
except ConnectionFailure as e:
    print("Could not connect to MongoDB")
    print(e)

MongoDB cluster is reachable
MongoClient(host=['ac-fx09sp5-shard-00-01.pnon21i.mongodb.net:27017', 'ac-fx09sp5-shard-00-02.pnon21i.mongodb.net:27017', 'ac-fx09sp5-shard-00-00.pnon21i.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='myAtlasClusterEDU', authsource='admin', replicaset='atlas-mrk941-shard-0', tls=True)


Acesse o banco de dados e a coleção:

In [17]:
mongo_client = MongoClient(uri)

DB_NAME="movies"
COLLECTION_NAME="movies_records"

db = mongo_client[DB_NAME]
collection = db[COLLECTION_NAME]

Para garantir que estamos trabalhando com uma coleção nova, exclua todos os registros existentes na coleção:

In [18]:

collection.delete_many({})


DeleteResult({'n': 100, 'electionId': ObjectId('7fffffff0000000000000098'), 'opTime': {'ts': Timestamp(1725375486, 1124), 't': 152}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1725375486, 1134), 'signature': {'hash': b'\xe2:%\xdd}\x92\x03\r\x08^\x9a4\x8e}\x8e\n\xcb\xb8\x04c', 'keyId': 7376317674717970437}}, 'operationTime': Timestamp(1725375486, 1124)}, acknowledged=True)

Neste passo, criamos um índice de busca vetorial na coleção `movies_records`, que será utilizado para garantir a recuperação eficiente de documentos no MongoDB. O índice pode ser criado usando o arquivo `vector-index.py` ou diretamente no MongoDB Atlas.

Em seguida, inicializamos um objeto de armazenamento vetorial (vector store) por meio do LlamaIndex e adicionamos os nodes a esse objeto utilizando o método `.add().`

In [19]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

vector_store = MongoDBAtlasVectorSearch(mongo_client, db_name=DB_NAME, collection_name=COLLECTION_NAME, index_name="vector_index")
vector_store.add(nodes)

index_name is deprecated. Please use vector_index_name
vector_index_name and index_name both specified. Will use vector_index_name


['c22e26f5-7b23-4f25-9cf9-28e4878c548a',
 '74a6b794-f5e0-4933-9838-5bbea2511ff3',
 '3b562c34-4f96-4470-b077-d14b914ae27a',
 'e6d4d105-0b09-48dc-bfa1-dc05bfc15280',
 '4dd37c83-f3b2-4d36-8aca-0bfa61f24bee',
 'decdc1df-b31d-4a90-a8e6-ef05d815b74d',
 '964de095-b627-4f0d-b82d-14f915e22b96',
 '0aa24efb-a10b-430a-8a59-5f1de6d4b18f',
 '040cb6c2-4311-4fa9-9a9c-340bc179b494',
 '01e0db6e-8a99-4b60-bbf7-628a55098108',
 'b9ae3a30-8438-4b7d-bcfe-f489c26ba760',
 '32065e55-3d53-4ac1-b3ff-eb899378e355',
 'bb48fcf3-1246-4689-8f19-fe72d85433e1',
 'aba1be89-6367-4699-b154-e6142919d7a0',
 'c6ff2681-e5de-4143-a25e-2285a8339d29',
 '61558c1c-ffcc-44ab-992a-9359713827d1',
 '4201f151-496a-4e6e-bb57-c4b8e2f2960a',
 '3ff8e751-569d-46a7-83b3-1a2a88629126',
 'b72a7164-a9fd-485d-9444-feada428ab18',
 '54569bf7-cebe-47eb-9c52-9d58746254cd',
 'f8f8d536-330c-41ee-b103-a1a75e636b71',
 '500a8f90-bb7b-4e6c-bc5c-25577aef7fe1',
 '020855af-7390-4c3e-b65b-da862aa8c34d',
 '0883db55-65b2-4f06-be30-5d4616a05777',
 'afa76012-4fbf-

O último passo é criar um mecanismo de consulta no LlamaIndex, que permite utilizar linguagem natural para recuperar informações relevantes e contextualmente apropriadas a partir do índice de dados.

In [23]:
import pprint
from llama_index.core.response.notebook_utils import display_response
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store)

query_engine = index.as_query_engine(similarity_top_k=3)
query = "Recomende um filme de fantasia para assistir com crianças e justifique sua escolha."
response = query_engine.query(query)
display_response(response)
pprint.pprint(response.source_nodes)


**`Final Response:`** "Jungle Book" seria uma ótimo filme de fantasia para assistir com crianças, pois conta a história de Mowgli, um menino criado por lobos que tenta se adaptar à vida na aldeia humana. A narrativa envolve aventuras emocionantes na selva, amizades incomuns com animais e lições valiosas sobre aceitação e compreensão. É uma escolha cativante e educativa para entreter e inspirar crianças de todas as idades.

[NodeWithScore(node=TextNode(id_='14b5723b-8003-4efd-bf3c-3e946c8ee4d3', embedding=None, metadata={'plot': 'Period piece about a Brazil that is no more. This movie is the sequel to "God and the Devil in the Land of the Sun" (Deus e o diabo na terra do sol), and takes place 29 years after Antonio ...', 'runtime': 100.0, 'genres': '["Action", "Crime", "Drama"]', 'fullplot': 'Period piece about a Brazil that is no more. This movie is the sequel to "God and the Devil in the Land of the Sun" (Deus e o diabo na terra do sol), and takes place 29 years after Antonio das Mortes killed Corisco (the "Blond Devil"), last of the Cangaceiros. In "the old days", Antonio\'s function in life was exterminate these bandits, on account of his personal grudges against them. His life had been meaningless for the last 29 years, but now, a new challenge awaits him. When a Cangaceiro appears in Jardim Das Piranhas, the local Land Baron (Jofre Soares), an old man, does what seems obvious to him: he calls Antoni